In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio

import os, h5py, sys
from pathlib import Path
import openpyxl

import holoviews as hv

In [ ]:
sys.path.append('../../src/')
from utility_code import RAS_Utility
ras = RAS_Utility()

In [ ]:
get_model_info('../../simulation/')

 ### get_manning_data

In [ ]:
# set location of hdf data
paln_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.p14.hdf'
geo_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.g03.hdf'
flow_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.u10.hdf'

In [ ]:
ras.get_manning_data(geo_hdf_loc).head()

 ### get_model_info
 
 This function needs to take the typical RAS folder structure and return the relevant plan associations for each hec ras plan

In [ ]:
# %cd ../../simulation

In [ ]:
# %pwd

In [ ]:
# path = %pwd
# path=Path(path)

In [ ]:
model_path = '../../simulation/'
model_path=Path(model_path)

In [ ]:
for child in Path(model_path).glob('*.prj'):
    if child.is_file():
        prj_file=child

prj_file

In [ ]:
prj_file_list=(Path.read_text(prj_file).split("\n"))
plan_list = [f'.{x.split("=")[-1]}' for x in prj_file_list if x.startswith("Plan File=")]
plan_list[:5]

In [ ]:
# find all files whose extension is in plan_list return the full path
plan_file_list = [x for x in Path(model_path).glob('*') if x.suffix in plan_list and x.stem != "Backup"]
plan_file_list

In [ ]:
# create a dictionary of plan names and file paths the name is found in the plan file
plan_dict = {}
for plan_file in plan_file_list:
    plan_name = Path.read_text(plan_file).split("\n")[0].split("=")[-1]
    plan_dict[plan_name] = plan_file

# create a dataframe of plan names and file paths
plan_df = pd.DataFrame.from_dict(plan_dict, orient='index', columns=['plan_file'])
plan_df

In [ ]:
# Path.read_text(plan_dict['002YR']).split("\n")

In [ ]:
# add the plans geometry file to plan_df
plan_df['geom_file'] = plan_df['plan_file'].apply(lambda x: Path.read_text(x).split("\n")[4].split("=")[-1])
plan_df.head()

In [ ]:
for x in plan_df['geom_file']:
    print(x)

In [ ]:
for x in plan_df['geom_file']:
    for y in Path(model_path).glob(f'*{x}'):
        if y.stem != "Backup":
            print(y)

In [ ]:
prj_file.stem

In [ ]:
# use glob to find the geometry file and add it to plan_df
plan_df['geom_file'] = plan_df['geom_file'].apply(lambda x: [y for y in Path(model_path).glob(f'*{prj_file.stem}.{x}')][0] if y.stem != "Backup" else None)

In [ ]:
plan_df

In [ ]:
for child in Path(model_path).glob('*.hdf'):
    if child.is_file():
        print(child)

In [ ]:
# check if plan file has an hdf file associated with it and add it to the dataframe
for x in Path(model_path).glob('*.p*.hdf'):
    print(x.stem)

In [ ]:
plan_df

In [ ]:
# add hdf file extension to geom_file if it exists
plan_df['geom_file_hdf'] = plan_df['geom_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)
plan_df

In [ ]:
# add hdf file extension to plan_file if it exists
plan_df['plan_file_hdf'] = plan_df['plan_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)
plan_df

In [ ]:
cTRL01_Laura2020_B = plan_df.loc['Laura 2020']

In [ ]:
cTRL01_Laura2020_B['plan_file']

In [ ]:
# define the function to get the file dataframe from the model path
def get_model_info(model_path):
    # find the projection file
    for child in Path(model_path).glob('*.prj'):
        if child.is_file():
            prj_file=child

    # read the project file and get the plan file extension
    prj_file_list=(Path.read_text(prj_file).split("\n"))
    plan_list = [f'.{x.split("=")[-1]}' for x in prj_file_list if x.startswith("Plan File=")]

    # find all files whose extension is in plan_list return the full path
    plan_file_list = [x for x in Path(model_path).glob('*') if x.suffix in plan_list]

    # create a dictionary of plan names and file paths the name is found in the plan file
    plan_dict = {}
    for plan_file in plan_file_list:
        plan_name = Path.read_text(plan_file).split("\n")[0].split("=")[-1]
        plan_dict[plan_name] = plan_file

    # create a dataframe of plan names and file paths
    plan_df = pd.DataFrame.from_dict(plan_dict, orient='index', columns=['plan_file'])

    # add the plans geometry file to plan_df
    plan_df['geom_file'] = plan_df['plan_file'].apply(lambda x: Path.read_text(x).split("\n")[4].split("=")[-1])

    # use glob to find the geometry file
    plan_df['geom_file'] = plan_df['geom_file'].apply(lambda x: [y for y in Path(model_path).glob(f'*{x}')][0])

    # add hdf file extension to geom_file if it exists
    plan_df['geom_file_hdf'] = plan_df['geom_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)

    # add hdf file extension to plan_file if it exists
    plan_df['plan_file_hdf'] = plan_df['plan_file'].apply(lambda x: Path(f'{x}.hdf') if Path(f'{x}.hdf').exists() else None)

    return plan_df

In [ ]:
get_model_info('../../simulation/')